In [ ]:
# %pip install webdriver-manager
# %pip install selenium
# %pip install html5lib

In [ ]:
# Neste caso, irei usar o Chrome

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

# Abrir o site(url)
driver.get(url)

In [ ]:
# Clicando no lenght da pagina em 100 itens/pagina
time.sleep(5)

button_100_itens = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", button_100_itens)

In [ ]:
# Pegando o número de páginas no HTML
numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")
numero_paginas = int(numero_paginas)
print(numero_paginas)

In [ ]:
lista_de_tabelas_por_pagina = []

for pagina in range(0, numero_paginas):

    # Lendo a tabela
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')

    # Pegando todo html
    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    display(tabela_final)

    lista_de_tabelas_por_pagina.append(tabela_final)

    avancar_pagina = driver.find_element("xpath", '''//*[@id="nextPage"]''')
    driver.execute_script("arguments[0].click()", avancar_pagina)

base_de_dados_completa = pd.concat(lista_de_tabelas_por_pagina)

display(base_de_dados_completa)

In [ ]:
# Trocando aba -> Performance

botao_aba = driver.find_element("xpath", '''//*[@id="table-tabs"]/li[2]/span''')
driver.execute_script("arguments[0].click();", botao_aba)

# Voltando a 1 página
for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)

In [ ]:
lista_de_tabelas_por_pagina = []

for pagina in range(0, numero_paginas):
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')
    html_tabela = tabela.get_attribute("outerHTML")
    tabela_final = pd.read_html(html_tabela)[0]
    # display(tabela_final)

    lista_de_tabelas_por_pagina.append(tabela_final)

    avancar_pagina = driver.find_element("xpath", '''//*[@id="nextPage"]''')
    driver.execute_script("arguments[0].click()", avancar_pagina)

base_de_dados_performance = pd.concat(lista_de_tabelas_por_pagina)
# display(base_de_dados_performance)

In [ ]:
driver.quit()

In [ ]:
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")
base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

base_de_dados_completa = base_de_dados_completa.set_index("Ticker")
# base_de_dados_completa = base_de_dados_completa[['Issuer', 'Expense Ratio', 'AUM']]

display(base_de_dados_performance, base_de_dados_completa)

In [ ]:
# Tabela FINAL
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)

display(base_de_dados_final)